In [ ]:
from sparkling.utils.gradient import get_kspace_loc_from_gradfile
from sparkling.utils.shots import convert_NCxNSxD_to_NCNSxD, convert_NCNSxD_to_NCxNSxD
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import warnings

In [ ]:
def normalize_frequency_locations(samples, Kmax=None):
    """
    This function normalizes the sample locations between [-0.5; 0.5[ for
    the non-Cartesian case.
    Parameters
    ----------
    samples: np.ndarray
        Unnormalized samples
    Kmax: int, float, array-like or None
        Maximum Frequency of the samples locations is supposed to be equal to
        base Resolution / (2* Field of View)
    Returns
    -------
    normalized_samples: np.ndarray
        Same shape as the parameters but with values between [-0.5; 0.5[
    """
    samples_locations = np.copy(samples.astype('float'))
    if Kmax is None:
        Kmax = np.abs(samples_locations).max(axis=0)
    elif isinstance(Kmax, (float, int)):
        Kmax = [Kmax] * samples_locations.shape[-1]
    Kmax = np.array(Kmax)
    samples_locations /= (2 * Kmax)
    if np.abs(samples_locations).max() >= 0.5:
        warnings.warn("Frequencies outside the 0.5 limit will be wrapped.")
        samples_locations = (samples_locations + 0.5) % 1 - 0.5
    return samples_locations

In [ ]:
def get_samples(filename, dwell_t, num_adc_samples, kmax):
    sample_locations = convert_NCxNSxD_to_NCNSxD(get_kspace_loc_from_gradfile(filename, dwell_t, num_adc_samples)[0])
    sample_locations = normalize_frequency_locations(sample_locations, Kmax=kmax)
    return sample_locations
def get_traj_params(traj_file):
    dico_params = get_kspace_loc_from_gradfile(traj_file)[1]
    FOV = dico_params['FOV']
    M = dico_params['img_size']
    Ns = dico_params['num_samples_per_shot']
    OS = dico_params['min_osf']
    return(FOV, M, Ns, OS)

In [ ]:
import scipy.io
ep = scipy.io.loadmat("/neurospin/lrmn/people/Zaineb_Amor/DataJuin2022/InvivoData_2302022_ep220179/Gfit_RS.mat")['gfit'][0,:]
at  = scipy.io.loadmat("/neurospin/optimed/ZainebAmor/InVivoData_JanuaryFebruaryMarch2022/Donnees10032022_invivo_at140305/Gfit_RS.mat")['gfit'][0,:]
cr = scipy.io.loadmat("/neurospin/lrmn/people/Zaineb_Amor/DataJuly2022/InVivo_cr170417_06072022/RS_HB_Gfit.mat")['gfit'][0,:]
cr_n = scipy.io.loadmat("/neurospin/lrmn/people/Zaineb_Amor/DataJuly2022/InVivo_cr170417_06072022/RS_Gfit.mat")['gfit'][0,:]
cr_mvt = scipy.io.loadmat("/neurospin/lrmn/people/Zaineb_Amor/DataJuly2022/InVivo_cr170417_06072022/RS_mvt_Gfit.mat")['gfit'][0,:]

In [ ]:
from scipy.signal import welch, periodogram
import matplotlib.pyplot as plt
import numpy as np
f1, P_first = welch(at[48:], fs = 1/(50*1e-3), scaling='spectrum', nperseg=48*10*3*2)
f2, P_sec = welch(ep[48:], fs = 1/(50*1e-3), scaling='spectrum', nperseg=48*10*3*2)
f3, P_third = welch(cr_n[48:], fs = 1/(50*1e-3), scaling='spectrum', nperseg=48*10*3*2)
f4, P_fourth = welch(cr[48:], fs = 1/(50*1e-3), scaling='spectrum', nperseg=48*10*3*2)
f5, P_fifth = welch(cr_mvt[48:], fs = 1/(50*1e-3), scaling='spectrum', nperseg=48*10*3*2)
plt.figure(figsize=(9*2.4,13))
plt.subplot(2,2,1)
plt.plot(f1,  np.sqrt(P_first),label='Volunteer#1')
plt.plot(f2, np.sqrt(P_sec),label='Volunteer#2')
plt.plot(f3, np.sqrt(P_third),label='Volunteer#3')
plt.legend(fontsize=12)
plt.xlabel('Hz', fontsize=15)
plt.ylabel('Power spectra in a.u', fontsize=15)
plt.xlim(0.012,0.4)
plt.ylim(0,2.6e-8)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title(r"(A) Power spectrum of $\Delta B_{0,dyn}$ collected at normal breathing - [0.12Hz, 0.4Hz]", fontsize=17)
plt.subplot(2,2,2)
plt.plot(f4, np.sqrt(P_fourth),label='Forced breathing')
plt.plot(f5, np.sqrt(P_fifth), label='Hand-to-chin movement')
plt.plot(f3, np.sqrt(P_third),label='Normal breathing')
plt.legend(fontsize=12)
plt.xlabel('Hz', fontsize=15)
plt.ylabel('Power spectra in a.u', fontsize=15)
plt.xlim(0.012,0.4)
plt.ylim(0,2.6e-8)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title(r"(B) Power spectra of $\Delta B_{0,dyn}$ collected from volunteer#3 - [0.12Hz, 0.4Hz]", fontsize=17)
plt.subplot(2,2,3)
plt.plot(f4, np.sqrt(P_fourth),label='Forced breathing')
plt.plot(f5, np.sqrt(P_fifth), label='Hand-to-chin movement')
plt.plot(f3, np.sqrt(P_third),label='Normal breathing')
plt.legend(fontsize=12)
plt.xlabel('Hz', fontsize=15)
plt.ylabel('Power spectra in a.u', fontsize=15)
plt.xlim(0.428,0.6)
plt.ylim(0,2.6e-8)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title(r"(C) Power spectra of $\Delta B_{0,dyn}$ collected from volunteer#3 - [0.43Hz, 0.6Hz]", fontsize=17)
plt.subplot(2,2,4)
plt.plot(f4, np.sqrt(P_fourth),label='Forced breathing')
plt.plot(f5, np.sqrt(P_fifth), label='Hand-to-chin movement')
plt.plot(f3, np.sqrt(P_third),label='Normal breathing')
plt.legend(fontsize=12)
plt.xlabel('Hz', fontsize=15)
plt.ylabel('Power spectra in a.u', fontsize=15)
plt.xlim(0.85,1.2)
plt.ylim(0,2.6e-8)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title(r"(D) Power spectra of $\Delta B_{0,dyn}$ collected from volunteer#3 - [0.85Hz, 1.2Hz]", fontsize=17)
plt.tight_layout()
plt.savefig("/neurospin/lrmn/people/Zaineb_Amor/DataJuly2022/spectralpow_3sub_hbm.png")

In [ ]:
from scipy.signal import welch, periodogram
import matplotlib.pyplot as plt
import numpy as np
f1, P_first = welch(at[48:], fs = 1/(50*1e-3), scaling='spectrum', nperseg=48*10*3*2)
f2, P_sec = welch(ep[48:], fs = 1/(50*1e-3), scaling='spectrum', nperseg=48*10*3*2)
f3, P_third = welch(cr_n[48:], fs = 1/(50*1e-3), scaling='spectrum', nperseg=48*10*3*2)
f4, P_fourth = welch(cr[48:], fs = 1/(50*1e-3), scaling='spectrum', nperseg=48*10*3*2)
f5, P_fifth = welch(cr_mvt[48:], fs = 1/(50*1e-3), scaling='spectrum', nperseg=48*10*3*2)
plt.figure(figsize=(10,14))
plt.subplot(2,1,1)
plt.plot(f1,  np.sqrt(P_first),label='Volunteer#1')
plt.plot(f2, np.sqrt(P_sec),'b', label='Volunteer#2')
plt.plot(f3, np.sqrt(P_third),label='Volunteer#3')
plt.legend(fontsize=12)
plt.xlabel('Hz', fontsize=15)
plt.ylabel('Power spectrum in a.u', fontsize=15)
plt.xlim(0.012,0.4)
plt.plot(f4, np.sqrt(P_fourth),label='Forced breathing')
plt.plot(f5, np.sqrt(P_fifth), label='Hand-to-chin movement' )
plt.legend(fontsize=12)
plt.xlabel('Hz', fontsize=15)
plt.ylabel('Power spectrum in a.u', fontsize=15)
plt.xlim(0.012,0.4)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title(r"Power spectrum of $\Delta B_{0,dyn}$", fontsize=17)